# Automatically translate template settings


## Setup google translate


### Use conda env on your machine.
The translation API does not work on VPN -- it returns the original en text. 
Maybe someone in FB is mocking the google trnslate api?

You can use these commands to setup new env:
```
conda create -n trans python=3.8
conda activate trans
pip install ipykernel
python -m ipykernel install --user --name=trans
```

### Install googletrans
```
pip install googletrans==3.1.0a
```

In [1]:
from googletrans import Translator, LANGUAGES
import time
import copy 

translator = Translator()

# Make sure you are not
test_text = 'Translation API works!'
res = translator.translate(test_text, dest="bg")
print(res.text)

assert res.text != test_text, "You are probably running this through the FB VPN!"

AttributeError: 'NoneType' object has no attribute 'group'

## Update the default template and langs

In [ ]:

en_template_settings = {
    # the last param is pre-processing function.
    "sentence1": ("Sentence 1: {sentence1}", None),
    "sentence2": ("Sentence 2: {sentence2}", None),
    "answer": "Sentence 1 and Sentence 2 have the same meaning. True or False? <mask>", # this needs to have <mask> which will be replaced with the label
    "label_map": {"true": "True", "false": "False"}
}

en_template_settings = { # the last param is pre-processing function.
            "sentence1": ("The fox eats meat, right?", None),
            #"sentence2": ("<mask>, {sentence2}", None),
            #"answer": "True, False, or Neither? <mask>",  # this needs to have <mask> which will be replaced with the label
            "label_map": {"entailment": "Yes, the fox eats rabbits!", 
                          "contradiction": "No, the fox do not eat rabbits",
                          "neutral": "Also, the fox eat eggs",
                          },
},

target_languages = ["de", "es", "fr", "ja", "ko", "zh"]
google_trans_languages_map = {"zh": "zh-cn"} # update this with the language

for lang in target_languages:
    trans_lang = google_trans_languages_map.get(lang, lang)
    
    if trans_lang not in LANGUAGES:
        similar_langs = [l for l in LANGUAGES.keys() if l.startswith(trans_lang)]
        print(f"'{trans_lang}' is not supported. Langs with this prefix: {similar_langs}")

In [ ]:
debug = False

translate_comment = "# translated with Google Translate"

def translate(txt, tgt_lang):
    tgt_lang = google_trans_languages_map.get(tgt_lang, tgt_lang)
    if debug:
        return f"{tgt_lang}: {txt} - translated"
    else:
        time.sleep(1) # to avoid blocking by Google Tranlate.
        # print(f"translator.translate('{txt}', dest={tgt_lang}).text")
        translated_text = translator.translate(txt, dest=tgt_lang).text
        #print(f"{translated_text}")
        return translated_text

def translate_template_settings(en_template_settings, tgt_lang):
    translated_settings = copy.deepcopy(en_template_settings)
    i = 0
    for k,v in translated_settings.items():
        if isinstance(v, str):
            translated_settings[k] = translate(v, tgt_lang)
            i += 1
        elif isinstance(v, tuple):
            new_tuple = []
            for vvi, vv in enumerate(v):
                if isinstance(vv, str):
                    vv = translate(vv, tgt_lang)
                new_tuple.append(vv)
            translated_settings[k] = tuple(new_tuple)
        elif isinstance(v, dict):
            for kk, vv in v.items():
                if isinstance(vv, str):
                    vv = translate(vv, tgt_lang)
                    i += 1
                v[kk] = vv

    return translated_settings


In [ ]:

translated_settings_multi = {}
translated_settings_multi["en"] = en_template_settings


In [ ]:
for lang in target_languages:
    if not lang in translated_settings_multi:
        print(f"Translating {lang}")
        res = translate_template_settings(en_template_settings, lang)
        translated_settings_multi[lang] = res


In [ ]:
# print config
print("{")
for k,v in translated_settings_multi.items():
    print(f"\t'{k}': {v},{('  ' + translate_comment) if k != 'en' else ''}")
print("}")
